In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

In [2]:
import os
import array

import pandas
import numpy

import plotly.express
import librosa
from matplotlib import pyplot as plt

from software.dataset.hussain2021 import load_data, load_meta
#from firmware.process import GravitySplitter

In [3]:

#dataset_path = './data/hx5kkkbr3j-1'
combined_path = '../data/hussain2021_brush_manual_s1.parquet'
samplerate = 50
lower = 2.0
upper = 5.0

In [4]:
acc_re = pandas.read_parquet(combined_path)
print(acc_re.columns)
data_columns = ['acc_x', 'acc_y', 'acc_z', 'elapsed']
index_columns = ['filename', 'time']
meta_columns = list(set(acc_re.columns) - set(data_columns) - set(index_columns))
acc_re

Index(['level_0', 'index', 'elapsed', 'acc_x', 'acc_y', 'acc_z', 'setting',
       'subject', 'session', 'gender', 'hand', 'sensor_location', 'brush',
       'location', 'sensor', 'label'],
      dtype='object')


level_0   index  \
filename                  time                                                
S11-S3-F-R-A-30-M-3-A.csv 2020-07-27 10:51:59.040000+00:00        0   18292   
                          2020-07-27 10:51:59.060000+00:00        1   18293   
                          2020-07-27 10:51:59.080000+00:00        2   18294   
                          2020-07-27 10:51:59.100000+00:00        3   18295   
                          2020-07-27 10:51:59.120000+00:00        4   18296   
...                                                             ...     ...   
S6-S5-M-R-A-30-M-2-A.csv  2020-08-06 05:11:18.440000+00:00  3021707  244543   
                          2020-08-06 05:11:18.460000+00:00  3021708  244544   
                          2020-08-06 05:11:18.480000+00:00  3021709  244545   
                          2020-08-06 05:11:18.500000+00:00  3021710  244546   
                          2020-08-06 05:11:18.520000+00:00  3021711  244547   

                                                              elapsed  \
filename                  time                                          
S11-S3-F-R-A-30-M-3-A.csv 2020-07-27 10:51:59.040000+00:00    0.00000   
                          2020-07-27 10:51:59.060000+00:00    0.01200   
                          2020-07-27 10:51:59.080000+00:00    0.03250   
                          2020-07-27 10:51:59.100000+00:00    0.05200   
                          2020-07-27 10:51:59.120000+00:00    0.07250   
...                                                               ...   
S6-S5-M-R-A-30-M-2-A.csv  2020-08-06 05:11:18.440000+00:00  163.25600   
                          2020-08-06 05:11:18.460000+00:00  163.27600   
                          2020-08-06 05:11:18.480000+00:00  163.29650   
                          2020-08-06 05:11:18.500000+00:00  163.31625   
                          2020-08-06 05:11:18.520000+00:00  163.33000   

                                                              acc_x    acc_y  \
filename                  time                                                 
S11-S3-F-R-A-30-M-3-A.csv 2020-07-27 10:51:59.040000+00:00  0.28200 -0.23900   
                          2020-07-27 10:51:59.060000+00:00  0.29325 -0.23725   
                          2020-07-27 10:51:59.080000+00:00  0.29375 -0.25825   
                          2020-07-27 10:51:59.100000+00:00  0.29725 -0.25800   
                          2020-07-27 10:51:59.120000+00:00  0.29975 -0.25225   
...                                                             ...      ...   
S6-S5-M-R-A-30-M-2-A.csv  2020-08-06 05:11:18.440000+00:00 -0.81250 -0.60725   
                          2020-08-06 05:11:18.460000+00:00 -0.79875 -0.62100   
                          2020-08-06 05:11:18.480000+00:00 -0.77125 -0.61150   
                          2020-08-06 05:11:18.500000+00:00 -0.73575 -0.63025   
                          2020-08-06 05:11:18.520000+00:00 -0.72400 -0.64500   

                                                              acc_z setting  \
filename                  time                                                
S11-S3-F-R-A-30-M-3-A.csv 2020-07-27 10:51:59.040000+00:00  0.93500    None   
                          2020-07-27 10:51:59.060000+00:00  0.95475    None   
                          2020-07-27 10:51:59.080000+00:00  0.96775    None   
                          2020-07-27 10:51:59.100000+00:00  0.97700    None   
                          2020-07-27 10:51:59.120000+00:00  0.96800    None   
...                                                             ...     ...   
S6-S5-M-R-A-30-M-2-A.csv  2020-08-06 05:11:18.440000+00:00  0.40375    None   
                          2020-08-06 05:11:18.460000+00:00  0.42250    None   
                          2020-08-06 05:11:18.480000+00:00  0.40625    None   
                          2020-08-06 05:11:18.500000+00:00  0.35725    None   
                          2020-08-06 05:11:18.520000+00:00  0.33300    None   

             

In [5]:
acc_re.label.value_counts(dropna=False)

label
None                     2864719
Left Lower Jaw Front       11632
Right Lower Jaw Front      11199
Right Upper Jaw Front      10963
Left Upper Jaw Front       10790
Left Lower Jaw Top         10416
Right Upper Jaw Back        9928
Left Upper Jaw Back         9910
Left Lower Jaw Back         9798
Right Lower Jaw Back        9635
Lower Incisors Front        9492
Left Upper Jaw Top          9258
Upper Incisors Front        9131
Right Lower Jaw Top         9033
Upper Incisors Back         8970
Right Upper Jaw Top         8623
Lower Incisors Back         8215
Name: count, dtype: int64

In [6]:
filename_meta =  acc_re.reset_index().groupby('filename').first()[meta_columns]
filename_meta.head(3)

,session,label,index,sensor,setting,brush,gender,hand,location,sensor_location,level_0,subject
filename,,,,,,,,,,,,
S11-S2-F-R-A-30-M-3-A.csv,2,Left Lower Jaw Front,9777,A,None,M,F,R,3,A,899600,S11
S11-S3-F-R-A-30-M-3-A.csv,3,Left Lower Jaw Front,18292,A,None,M,F,R,3,A,0,S11
S11-S4-F-R-A-30-M-3-A.csv,4,Left Lower Jaw Front,28741,A,None,M,F,R,3,A,2318000,S11


In [7]:
acc_re.reset_index().filename.nunique()

30

In [8]:
import seaborn

In [ ]:

import pandas as pd
import numpy as np

def plot_spans(ax, df, time_col, label_col, colors=None, alpha=0.3):
    if colors is None:
        unique_labels = df[label_col].unique()
        colors = dict(zip(unique_labels, plt.cm.tab10(np.linspace(0, 1, len(unique_labels)))))
    
    # Group consecutive rows with same label
    df = df.copy()
    df = df.dropna(subset=label_col) # avoid NaNs
    #df = df.fillna(subset=label_col, 'none')
    df['group'] = (df[label_col] != df[label_col].shift()).cumsum()

    groups = df.groupby('group')
    print(len(groups))
    
    for _, group in groups:
        label = group[label_col].iloc[0]
        start = group[time_col].iloc[0]
        end = group[time_col].iloc[-1]
        ax.axvspan(start, end, alpha=alpha, color=colors[label], label=label)
    
    # Remove duplicate legend entries
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    

def plot_timeline(df, sr=50, columns=None):
    if columns is None:
        columns = ['acc_x', 'acc_y', 'acc_z']
    df = df.sort_values('time')
    df = df.reset_index()
    df = df.dropna(subset=columns)
    df = df.iloc[0:20000]
    
    #print(df.label.unique())
    print(df.head())
    
    df['elapsed'] = (df['time'] - df['time'].min()) / pandas.Timedelta(seconds=1)

    fig, ax = plt.subplots(1, figsize=(20, 5))
    for c in columns:
        seaborn.lineplot(ax=ax, data=df, x='elapsed', y=c, lw=0.5, alpha=0.5, label=c)
        #ax.plot(

    plot_spans(ax=ax, df=df, time_col='elapsed', label_col='label')
    
    return fig

grouped = acc_re.groupby('filename')
for idx, g in grouped:
    plot_timeline(g)
    break


                    filename                             time  level_0  index  \
0  S11-S2-F-R-A-30-M-3-A.csv 2020-07-25 11:52:33.800000+00:00   899600   9777   
1  S11-S2-F-R-A-30-M-3-A.csv 2020-07-25 11:52:33.800000+00:00   959205   9777   
2  S11-S2-F-R-A-30-M-3-A.csv 2020-07-25 11:52:33.800000+00:00   976235   9777   
3  S11-S2-F-R-A-30-M-3-A.csv 2020-07-25 11:52:33.800000+00:00   950690   9777   
4  S11-S2-F-R-A-30-M-3-A.csv 2020-07-25 11:52:33.800000+00:00   984750   9777   

   elapsed   acc_x   acc_y   acc_z setting subject  session gender hand  \
0   0.0096  0.1958 -0.4688  0.8624    None     S11        2      F    R   
1   0.0096  0.1958 -0.4688  0.8624    None     S11        2      F    R   
2   0.0096  0.1958 -0.4688  0.8624    None     S11        2      F    R   
3   0.0096  0.1958 -0.4688  0.8624    None     S11        2      F    R   
4   0.0096  0.1958 -0.4688  0.8624    None     S11        2      F    R   

  sensor_location brush location sensor label  
0             

In [ ]:
def make_spectrogram(df, sr=50, hop_length=16, columns=None):
    if columns is None:
        columns = ['acc_x', 'acc_y', 'acc_z']
    df = df.sort_values('time')
    df = df.dropna(subset=columns)

    n_fft = 4*hop_length

    # Make combined XYZ spectrogram, concat on frequency axis
    ss = []
    for c in columns:
        v = df[c].values
        #print(v.shape)
        S = librosa.stft(v, n_fft=n_fft, hop_length=hop_length)
        S = numpy.abs(S)
        S = librosa.amplitude_to_db(S)
        S = S[0:n_fft//4, :]
        S = S[1:,:] # drop DC
        #print(S.shape)
        ss.append(S)

    #raise Exception('ff')
    o = numpy.concatenate(ss)
    #print('o', o.shape)
    return o

mm = acc_re
grouped = mm.groupby('filename')
hop_length=32
for idx, g in grouped:
    make_spectrogram(g, hop_length=hop_length)
    break
specs = grouped.apply(make_spectrogram, include_groups=False, hop_length=hop_length).to_frame().rename(columns={0:'spectrogram'})
specs;

In [ ]:

for filename, spec in specs.iterrows():
    #print(filename)
    fig, ax = plt.subplots(1, figsize=(20, 8))
    S = spec.spectrogram
    #S = S[:, 0:400]
    N = numpy.quantile(S, q=0.10, axis=1, keepdims=True)
    #S = S - N
    #print(S.shape, N.shape)
    librosa.display.specshow(ax=ax, data=S)


In [ ]:
def to_spectrum(S):
    s = numpy.mean(S, axis=1)
    o = pandas.Series(s, index=numpy.arange(len(s)))
    return o

def compute_spectrums(df, sr, metric='motion', axes=['x', 'y', 'z']):
    dfs = []
    
    for axis in axes:
        c = f'{metric}_{axis}'

        freqs = librosa.fft_frequencies(n_fft=4*16, sr=50)
    
        specs = df.groupby('filename').apply(make_spectrogram, include_groups=False, columns=[c]).to_frame().rename(columns={0:'spectrogram'})
        ss = specs.spectrogram.apply(to_spectrum)
        ss.columns = freqs

        # Normalize
        N = numpy.mean(ss.values, axis=1, keepdims=1)
        ss = pandas.DataFrame(ss-N, columns=ss.columns, index=ss.index)
        
        sf = pandas.melt(ss.reset_index(), id_vars='filename', var_name='frequency')
        sf['axis'] = axis
        
        dfs.append(sf)

    out = pandas.concat(dfs)
    return out

spectrums = compute_spectrums(mm, sr=samplerate, metric='acc')
#spectrums

fig = plotly.express.line(spectrums,
    facet_row='axis',
    color='filename',
    x='frequency',
    y='value',
    height=600,
    width=1000,
    title='Toothbrushing spectrum - Hussain 2021 dataset',
    labels={
        "frequency": "Frequency (Hz)",
        "value": "Energy (dB)",
     },
)

fig.add_vrect(x0=lower, x1=upper, line_width=0, fillcolor="green", opacity=0.2, layer='below')
fig.update_layout(showlegend=False)